In [2]:
from collections import defaultdict
import pprint
import re
import pandas as pd
import numpy as np

In [3]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "cambridge.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 5 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write(bytes('<?xml version="1.0" encoding="UTF-8"?>\n',encoding='utf-8'))
    output.write(bytes('<osm>\n  ',encoding='utf-8'))

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(bytes('</osm>',encoding='utf-8'))

In [4]:
OSMFILE = SAMPLE_FILE


expected = ["02138", "02139", "02140", "02141", "02142", "02238"]


#Create list of mapping keys
mapping_keys = []
for k,v in mapping.items():
    mapping_keys.append(k)


def audit_state(state_types, state_name):
    if state_name not in expected:
        state_types[state_name].add(state_name)


def is_state(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit(osmfile):
    osm_file = open(osmfile, "r",encoding='utf8')
    state_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file,events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_state(tag):
                    audit_state(state_types, tag.attrib['v'])
    osm_file.close()
    return state_types


def update_name(name, mapping):
    if name in mapping_keys: #If the bad key is in the mapping dictionary, then perform a substitute, otherwise leave as-is
        good = mapping[name]
        return good
    else:
        return name
        
def st_types(file):
    st_types = audit(file)
    pprint.pprint(dict(st_types))

def test_w_update(file):
    st_types = audit(file)
    pprint.pprint(dict(st_types))
    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print(name, "=>", better_name)


In [5]:
st_types(SAMPLE_FILE)

{'01944': {'01944'},
 '02108': {'02108'},
 '02109': {'02109'},
 '02110': {'02110'},
 '02110-1301': {'02110-1301'},
 '02111': {'02111'},
 '02114': {'02114'},
 '02115': {'02115'},
 '02116': {'02116'},
 '02118': {'02118'},
 '02128': {'02128'},
 '02129': {'02129'},
 '02134': {'02134'},
 '02134-1305': {'02134-1305'},
 '02134-1306': {'02134-1306'},
 '02134-1307': {'02134-1307'},
 '02134-1316': {'02134-1316'},
 '02134-1317': {'02134-1317'},
 '02134-1318': {'02134-1318'},
 '02134-1319': {'02134-1319'},
 '02134-1321': {'02134-1321'},
 '02134-1433': {'02134-1433'},
 '02134-1442': {'02134-1442'},
 '02135': {'02135'},
 '02138-2903': {'02138-2903'},
 '02143': {'02143'},
 '02144': {'02144'},
 '02145': {'02145'},
 '02149': {'02149'},
 '02155': {'02155'},
 '02215': {'02215'},
 '02446': {'02446'},
 '02472': {'02472'},
 '02474': {'02474'},
 '02478': {'02478'}}


The interesting thing here is that there are many zip codes that are not actually in Cambridge. For example, I know 02108 is in Boston proper (I used to live there). However, reflecting on the XML file I downloaded from Mapzen, I recall that it wasn't exclusively Cambridge. Instead, it was a rectangular area that encompassed Cambridge as well as some areas in Boston. I'll need to add Boston zip codes to the expected list to account for that.

In [8]:
expected =  ['02118',
    '02119',
    '02120',
    '02130',
    '02134',
    '02135',
    '02445',
    '02446',
    '02447',
    '02467',
    '02108',
    '02114',
    '02115',
    '02116',
    '02215',
    '02128',
    '02129',
    '02150',
    '02151',
    '02152',
    '02124',
    '02126',
    '02131',
    '02132',
    '02136',
    '02109',
    '02110',
    '02111',
    '02113',
    '02121',
    '02122',
    '02124',
    '02125',
    '02127',
    '02210',
    '02138',
    '02139',
    '02140',
    '02141',
    '02142',
    '02238']

Let's try running that again to see if there are any unexpected zip codes now.

In [9]:
st_types(SAMPLE_FILE)

{'01944': {'01944'},
 '02110-1301': {'02110-1301'},
 '02134-1305': {'02134-1305'},
 '02134-1306': {'02134-1306'},
 '02134-1307': {'02134-1307'},
 '02134-1316': {'02134-1316'},
 '02134-1317': {'02134-1317'},
 '02134-1318': {'02134-1318'},
 '02134-1319': {'02134-1319'},
 '02134-1321': {'02134-1321'},
 '02134-1433': {'02134-1433'},
 '02134-1442': {'02134-1442'},
 '02138-2903': {'02138-2903'},
 '02143': {'02143'},
 '02144': {'02144'},
 '02145': {'02145'},
 '02149': {'02149'},
 '02155': {'02155'},
 '02472': {'02472'},
 '02474': {'02474'},
 '02478': {'02478'}}


We have two majors types of issues: (2) nine-digit zip codes, and (2) five-digit zip codes that were not part of our list. For (1) we need to update our function to handle this special case, and extract only the first five digits. 

In [26]:
def update_name(name, mapping):
    if len(name) == 10: #If the zip is 9-digit format, take the left-most five digits
        return name[0:5]
    else:
        return name

Now that we've taken care of 9-digit zip codes, we need to see what's going on with a few of the other zip codes that aren't in our list. I'm not sure if all of the towns represented are part of the Great Boston area. We can check that by looking at a list of the cities that are represented in the list. 

In [51]:
cities_dict = defaultdict(int)
for event, elem in ET.iterparse(SAMPLE_FILE,events=("start",)):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if tag.attrib['k'] == "addr:city":
                cities_dict[tag.attrib['v']] += 1 #Create a dictionary of city names included in 
print(cities_dict)

defaultdict(<class 'int'>, {'Boston': 162, 'Cambridge': 117, '2067 Massachusetts Avenue': 1, 'Somerville': 54, 'boston': 1, 'Brighton': 4, 'Watertown': 3, 'Watertown, MA': 1, 'Arlington': 33, 'Allston': 9, 'Charlestown': 4, 'Everett': 2, 'Medford': 4, 'Arlington, MA': 1, 'Belmont': 2, 'Cambridge, Massachusetts': 1, 'Boston, MA': 1, 'Brookline': 1})


It looks like all of the cities represented are cities that could be called part of "Greater Boston," so let's keep them. 

In [54]:
def update_zip(name, mapping):
    elif len(name) == 10: #If the zip is 9-digit format, take the left-most five digits
        return name[0:5]
    else:
        return name